### Chain Reaction - Minimum Bombs Needed (Extreme Version)
##### Codewars | 2 kyu | 65c420173817127b06ff7ea7

Given a string that represents a 2D grid, return the minimum number of bombs needed to be set off to destroy all bombs in the grid. A grid contains an empty space "0"; a bomb that explodes directly above, below, left, and right ("+"); and a bomb that explodes inn all four diagonal directions ("x").

The map of size ```N = width x height``` needs to run in time ```O(N og N)``` at a maximum. A ```O(N)``` soution is possible but not required.

- 10 fixed maps (up to 5 x 5)
- 100 small random maps (10 x 10)
- 50 medium random maps (50 x 50)
- 10 large random maps (100 x 100)
- 2 giant random maps (250 x 250)


#### Approach 1

We cannot use a disjoinnt set because the relationships between bombs can be asymmetric. We also are not looking for any strongly or weakly connected subgraphs. What we can do is use DFS/BFS to traverse nodes and 'color' them. We start at a bomb and 'color' until there is nothing left. We can color over past traversals. We keep track of how many colors remain in a list (color is index and number of tiles is entry). At the end, the number of non-zero colors inndicates the number of bombs we must set off.

**1st Att** -> 42/50 Medium Random Maps

In [91]:
def min_bombs_needed(grid):
    '''
    O(N^2) Time Complexity
    '''

    def dfs_color(i, j, c):

        # Base case: if in empty space or aready traversed in this pass
        if (grid[i][j] == '0') or (cgrid[i][j] == c):
            return
        
        # If colored, will remove one from clist color totals
        # Adds one to total of current color c
        clist[cgrid[i][j]] -= 1
        clist[c] += 1
        cgrid[i][j] = c

        if grid[i][j] == "+":
            dfs_color(max(0, i - 1), j, c)
            dfs_color(min(len(grid) - 1, i + 1), j, c)
            dfs_color(i, max(0, j - 1), c)
            dfs_color(i, min(len(grid[0]) - 1, j + 1), c)

        else: # x
            dfs_color(i - 1, j - 1, c) if (i > 0) and (j > 0) else ''
            dfs_color(i - 1, j + 1, c) if (i > 0) and (j < len(grid[0]) - 1) else ''
            dfs_color(i + 1, j - 1, c) if (i < len(grid) - 1) and (j > 0) else ''
            dfs_color(i + 1, j + 1, c) if (i < len(grid) - 1) and (j < len(grid[0]) - 1) else ''

    # String to 2D list
    grid = [list(s) for s in grid.split('\n')]

    # Stores colors of each bomb
    cgrid, clist = [[0 for _ in range(len(grid[0]))] for _ in range(len(grid))], [0]

    # Traverse across grid and call DFS 
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if (not cgrid[i][j]) and (grid[i][j] in ['+', 'x']):
                clist += [0]
                dfs_color(i, j, len(clist) - 1)
    
    # A bomb needs to be set off for each color
    return sum(z > 0 for z in clist)   

In [92]:
t = "0+000x000xx\n" + "++000++000+\n" + "00x0x00000x\n" + "0+000000x+0"

min_bombs_needed(t)

3

#### Approach 2

Isolate SCCs and their connections to create a Condensed DAG, from there we pick a node, preform DFS to see how many other SCCs we explode, and do a reverse DFS to see how many SCC nodes can explode us. Repeat until all nodes in the condensated DAG have been explored and return the number of times we need to preform DFS. 

1. Create the graph from the input string -> O(n)
    1. Must create an adjancency list as well as record the number of nodes
2. Preform Tarjan's SCC Algorithm to get all SCCs -> O(n)
3. Traverse the Condensed DAG and record how many DFS calls are needed -> O(n)

**NOTE:** When we preform the reverse DFS, when we backtrack we also need to call DFS on the node we backtracked to because it can explode more than just the one bomb we backtracked from

In [93]:
from collections import defaultdict

def create_adj_matrix_depreciated(g: str) -> tuple:
    '''
    O(N) Time Complexity

    Input: 
        g | string format of the 2D grid
    Output:
        2D list | Adjancency Matrix
        int | Number of Nodes
    '''
    p = [list(s) for s in g.split('\n')]

    x_ = [[1, 1], [-1, -1], [1, -1], [-1, 1]]
    p_ = [[1, 0], [-1, 0], [0, 1], [0, -1]]

    d = defaultdict(lambda: 0) # Records (coords : id, type) pairs
    id = 0 # id iterator & number of nodes

    # Searches for all nodes in input list
    for i in range(len(g)):
        for j in range(len(g[0])):
            if g[i][j] in ['x', '+']:
                d[(i, j)] = [id, g[i][j]]
                id += 1
    
    # Adj matrix
    m = [[0 for _ in range(id)] for _ in range(id)]

    # Since we know how bombs connect, iterate through
    # dictionary and can use O(1) lookups to see if two bombs connect
    for p in sorted(list(d)):

        # Get all info of current node
        v, i, j = d[p], p[0], p[1]
        id_, ty = v[0], v[1]

        pos = x_ if ty == 'x' else p_

        # Based on type and surroundings will
        # add to adjacency matrix
        for x, y in pos:
            cur = d[(i + x, j + y)]
            if cur:
                m[id_][cur[0]] = 1

    return m, id


In [94]:
def condense_graph(adj: list[list[int]], low_link: list[int], n_scc: int, n_nodes: int):
    ''' 
    O(N) Time Complexity

    Condense graph then preform min_bombs on it and search DFS and reverse.
    Because it is a DAG, no risk of finding a local optimum or incorrect solution.

    Input:
        adj - 2D Python Adjacency Matrix
        low_link - low_link values for nodes in order of how they appear on graph L -> R, U -> D
        n_scc - Number of strongly connected components
        n_nodes - Number of nodes on original graph
    '''

    o = dict(zip(set(low_link), range(n_scc))) # From low_link values to new nodes
    adj_nu = [[0 for _ in range(n_scc)] for _ in range(n_scc)] # New adjacency matrix
    v = [0 for _ in range(n_nodes)] # Tracks visited nodes

    def dfs_condensed(at):
        v[at] = 1
        at_nu = o[low_link[at]]
        for ind, c in zip(adj[at], range(n_nodes)):
            if ind == 1: # If there exists a connection
                c_nu = o[low_link[c]]
                if at_nu != c_nu: # If not apart of the same SCC add connection to new adj matrix
                    adj_nu[at_nu][c_nu] = 1 
                if not v[c]:
                    dfs_condensed(c)
                
    for i in range(n_nodes):
        if not v[i]:
            dfs_condensed(i)

    return adj_nu

---

In [95]:
from collections import defaultdict

def create_adj_matrix(g: str) -> tuple:
    '''
    O(N) Time Complexity

    Input: 
        g | string format of the 2D grid
    Output:
        2D list | Adjancency Matrix
        int | Number of Nodes
    '''

    # Bomb directions
    x_ = [[1, 1], [-1, -1], [1, -1], [-1, 1]]
    p_ = [[1, 0], [-1, 0], [0, 1], [0, -1]]
    
    id = 0 # id iterator & number of nodes
    d = defaultdict(lambda: 0) # Records (coords : id, type) pairs
    
    # Parse string and add all nodes to dict
    i, w, l = 0, 0, 0
    while i < len(g):
        if g[i] in ['x', '+']:
            d[(l, w)] = [id, g[i]]
            id += 1
        elif g[i] == '\n':
            w = -1
            l += 1
        w += 1
        i += 1
        
    # Adjancency Matrix
    m = [[0 for _ in range(id)] for _ in range(id)]

    # Fill In
    for p in list(d):

        # Get all info of current node
        v, i, j = d[p], p[0], p[1]
        id_, ty = v[0], v[1]

        pos = x_ if ty == 'x' else p_

        # Based on type and surroundings will
        # add to adjacency matrix
        for x, y in pos:
            cur = d[(i + x, j + y)]
            if cur:
                m[id_][cur[0]] = 1

    return m, id


In [96]:
def tarjan_scc(g: list[list[int]], n: int) -> int:
    '''
    O(N) Time Complexity
    
    Input: 
        n -> number of nodes in the graph
        g -> 2D list; adjacency matrix
    Output:
        int -> Number of 'source' nodes
            -> Source Node: node without incoming edges in condensed graph
            -> Since a DAG, num source nodes = num starts needed to detonate all bombs
    '''

    def dfs_tarjan(at: int) -> None:
        '''
        Input:
            at | int | ID of the node we are currently at

        No output as we modify information-tracking objects
        ids, low, onStack, and stack
        '''
        global id, sccCount
        stack.append(at) 
        onStack[at] = True
        id += 1
        ids[at] = low[at] = id

        # Visit all neighbours & min low-link on callback
        for x, y in zip(g[at], range(n)):
            if x == 1:
                if ids[y] == -1:
                    dfs_tarjan(y)
                if onStack[y]:
                    low[at] = min(low[at], low[y])
                else:
                    inc[low[y]] = True

        # After visit all neighbours, if we are at start
        # of a SCC, empty the seen until we at start of SCC
        if ids[at] == low[at]:
            while stack:
                node = stack.pop(-1)
                onStack[node] = False
                low[node] = sccCount
                if node == at:
                    break
            sccCount += 1

    global id, sccCount
    id = 0 # Used to give each node an id
    sccCount = 0 # Used to count number of SCCs found

    # Index i represents node i
    ids = [-1 for _ in range(n)]
    low = [0 for _ in range(n)]
    onStack = [False for _ in range(n)]

    # At most n SCCs exist, tracks if an SCC has an incoming edge 
    # Works because SCCs removed from stack in reverse topological order
    inc = [False for _ in range(n)]

    stack = []

    for i in range(n):
        if ids[i] == -1:
            dfs_tarjan(i)
    
    return sccCount - sum(inc[:sccCount + 1])

In [97]:
def min_bombs_needed(grid):
    adj, n_node = create_adj_matrix(grid)
    if not n_node: return 0 
    return tarjan_scc(adj, n_node)

In [98]:
test_grid = \
    "0+000x000xx\n" + \
    "++000++000+\n" + \
    "00x0x00000x\n" + \
    "0+000000x+0"

test_grid2 = \
    "x+xxxx0x00\n" + \
    "0+x00+00+x\n" + \
    "+xx+++++++\n" + \
    "x0++00x+0+\n" + \
    "xx0xx+0x+x\n" + \
    "00x+0xx+++\n" + \
    "+x0+x000+x\n" + \
    "xx+0++0+0+\n" + \
    "x+xx0+0x0+\n" + \
    "x00+0+0+x+"

test_grid3 = \
    "0++++x000+\n" + \
    "+x0x+x+0+x\n" + \
    "x0+00x+00x\n" + \
    "++00++++0+\n" + \
    "00+0++++0x\n" + \
    "x0x+++++0+\n" + \
    "+0x++0+0++\n" + \
    "+xx0+0000x\n" + \
    "+0+++++0x0\n" + \
    "x+x++0000x"

min_bombs_needed(test_grid3)

6